In [1]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI
from whisper.utils import system_encoding

In [2]:
## .env 설정

load_dotenv()

True

In [3]:
from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableSequence, RunnableLambda, RunnableParallel

# LangChain OpenAI
from langchain_openai import ChatOpenAI

## 이미 생성된 합성 데이터셋 불러오기

In [4]:
## 이미 생성된 합성 데이터셋 생성
import pandas as pd

# 같은 폴더(현재 작업 디렉토리)에 저장했다면
dataset_df = pd.read_csv("./ragas_dataset1.csv")
print(dataset_df.shape)
display(dataset_df)

(12, 4)


,user_input,reference_contexts,reference,synthesizer_name
0,크랙(Crack) 앱의 주요 기능은 무엇인가요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"크랙(Crack)은 AI 페르소나 챗봇 앱으로, 코드노바의 서비스 아키텍처에서 생성...",single_hop_specific_query_synthesizer
1,RabbitMQ의 역할은 무엇인가요?,"['### 2.2. 데이터베이스\n- **역할**: 사용자 데이터, 콘텐츠, 로그 ...",RabbitMQ는 비동기 작업 처리를 위해 시스템 간 메시지를 전달하는 메시지 큐입니다.,single_hop_specific_query_synthesizer
2,프론트엔드 팀에서 JWT의 역할은 무엇인가요?,['### 2.5. 파일 저장소\n- **역할**: 이미지 및 기타 미디어 파일을 ...,JWT(JSON Web Token)는 사용자 인증을 처리하는 데 사용됩니다.,single_hop_specific_query_synthesizer
3,"코드노바의 백엔드 서비스 아키텍처는 어떻게 설계되었고, 그 목적은 무엇인가요?",['## 5. 모니터링 및 로깅\n- **모니터링 도구**: Prometheus 및...,코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 ...,single_hop_specific_query_synthesizer
4,MFA는 뭐하는거야?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,MFA는 추가적인 보안 강화를 위해 도입되는 다중 인증 방법입니다.,single_hop_specific_query_synthesizer
5,JWT는 API 접근 시 어떤 역할을 합니까?,"['### 2.2 비밀번호 관리\n- 비밀번호는 최소 8자 이상, 대문자, 소문자,...","JWT(JSON Web Token)는 API 접근 시 인증을 위해 사용되며, 토큰은...",single_hop_specific_query_synthesizer
6,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 무엇입니까?,['## 4. 로그 관리\n\n### 4.1 로그 기록\n- 모든 인증 시도 및 A...,이 가이드는 코드노바의 백엔드 개발팀이 보안 및 인증을 효과적으로 관리하기 위한 기...,single_hop_specific_query_synthesizer
7,2025년 8월 29일에 작성된 코드노바의 에러 핸들링 매뉴얼에서 에러 핸들링의 중...,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,코드노바의 에러 핸들링 매뉴얼에서는 에러 핸들링이 안정적인 시스템 운영을 위해 필수...,single_hop_specific_query_synthesizer
8,v1.0 뭐에요?,['<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-2...,"v1.0은 코드노바 백엔드팀의 배포 및 운영 가이드의 버전 번호로, 이 문서는 안정...",single_hop_specific_query_synthesizer
9,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,['## 체크리스트\n\n- [ ] 코드 리뷰 완료\n- [ ] 모든 테스트 통과\...,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,single_hop_specific_query_synthesizer


## 우리가 쓰는 sllm RAG 체인 불러오기

In [5]:
from dotenv import load_dotenv

load_dotenv()

from pathlib import Path
import sys

root = Path.cwd()
while not (root / "utils2").exists() and root != root.parent:
    root = root.parent
sys.path.insert(0, str(root))

from utils2.rag import chat

response = chat({"history": [
    {"role": "user", "content": "코드노바의 API 서버 기술스택알려줘"}], "tone":"informal", "permission":"cto"})

INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: BAAI/bge-m3
INFO:utils2.service:Chat Request - Permission: cto, Tone: informal
INFO:utils2.service:Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다. 질문이 들어오면 반드시 cto_search 툴을 호출하여 문서를 검색하세요.
INFO:httpx:HTTP Request: POST https://qfs7408cldsqap-8000.proxy.runpod.net/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:LLM Tool Parse Response Success
INFO:utils2.service:LLM Tools Match
INFO:utils2.service:['{"name": "cto_search", "arguments": {"keyword": "코드노바의 API 서버 기술스택알려줘"}}']
INFO:utils2.service:cto search keyword: 코드노바의 API 서버 기술스택알려줘
INFO:chromadb.telemetry.product.posthog:Anonymized telemetry enabled. See                     https://docs.trychroma.com/telemetry for more information.


name='cto_search' description='모든 team(backend, frontend, data_ai)의 문서를 검색합니다.' args_schema=<class 'langchain_core.utils.pydantic.cto_search'> func=<function cto_search at 0x000001B235D6EFC0>


INFO:utils2.service:cto search found 5 results.
INFO:utils2.service:cto_search tool Response Success


검색 결과:
-----
<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 서비스 아키텍처 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 개요
코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.

## 2. 아키텍처 구성 요소

### 2.1. API 서버
- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.
- **기술 스택**: Node.js, Express.js
- **검증 포인트**:
  - API 엔드포인트가 올바르게 작동하는지 확인
  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]
<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# API 정책 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 목적
이 문서는 코드노바의 백엔드팀에서 API를 설계하고 운영하는 데 필요한 정책과 절차를 정의합니다. API의 일관성과 안정성을 보장하고, 외부 개발자 및 내부 팀 간의 원활한 협업을 촉진하기 위한 것입니다. [[ref2]]
<!-- 회사: 코드노바 | 대상: 사원(프론트엔드) | 작성일: 2025-08-29 -->
# 프론트엔드 아키텍처 문서
분류: frontend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

---

## 1. 개요
코드노바의 프론트엔드 아키텍처는 사용자 경험을 극대화하고, 유지보수성을 높이며, 확장성을 고려하여 설계되었습니다. 본 문서는 프론트엔드 개발자가 이해하고 실행할 수 있도록 아키텍처의 구성 요소와 원칙을 설명합니다.

## 2. 아키텍처 구성 

INFO:httpx:HTTP Request: POST https://qfs7408cldsqap-8000.proxy.runpod.net/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:Final Assistant Reply Generated
INFO:utils2.service:코드노바의 API 서버 기술스택은 Node.js와 Express.js로 구성되어 있습니다. 이 API 서버는 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환하는 역할을 합니다.


## 정성적 평가 체인 셋팅

In [10]:
def api_document_evaluation_chain_json():
    llm = ChatOpenAI(
        model="gpt-4o",
        temperature=0,
        model_kwargs={"response_format": {"type": "json_object"}},
    )

    evaluation_prompt = PromptTemplate.from_template(
        """
    당신은 코드노바의 사내 문서 기반 평가 전문가입니다. 주어진 시스템의 답변이 **기준 정답과 얼마나 일치하는지**, **중요한 정보를 얼마나 잘 재현했는지**, 그리고 **세부 사항을 얼마나 잘 반영했는지** 평가해야 합니다.

    평가 기준:
    1. **정확도 (Accuracy)**: 시스템의 답변이 **기준 정답**과 얼마나 일치하는지 평가하세요.
        - 100점: **완벽하게 일치**. 구체적인 매개변수, 반환값, 예시까지 정확히 일치.
        - 90-99점: **일치**하나, 예시나 코드에서 **약간 차이**가 있을 때.
        - 70-89점: **핵심 내용**은 맞지만 **세부 내용**이 **누락되거나 잘못된 설명**.
        - 0-69점: **기준 정답과 크게 다름**. 예시나 코드에서 **큰 차이**가 있을 경우 점수는 **0점**.

    2. **재현율 (Recall)**: 시스템의 답변이 **기준 정답에서 제공한 중요한 정보**(예: 매개변수, 반환값 등)를 얼마나 잘 재현했는지 평가하세요.
        - 100점: **모든 핵심 정보**를 **정확하게 재현**.
        - 90-99점: **핵심 정보**는 잘 재현되었으나 **세부 내용**이 부족하거나 다르게 표현됨.
        - 70-89점: **핵심 정보**가 **누락**되었거나 **잘못된 설명**이 있음.
        - 0-69점: **핵심 정보**가 많이 **누락**됨.

    3. **구체성 (Specificity)**: 시스템의 답변이 **기준 정답에서 제공한 세부 사항**(매개변수 설명, 예시 코드 등)을 얼마나 구체적으로 반영했는지 평가하세요.
        - 100점: **매개변수**와 **반환값**을 **구체적으로 설명**하며, **예시 코드**나 **상세한 설명**을 포함.
        - 90-99점: **핵심 매개변수**는 설명되었지만, **구체적인 형식이나 예시 코드**가 부족.
        - 70-89점: **기본적인 설명**은 있지만 **구체적인 예시나 설명**이 부족하거나 불완전.
        - 0-69점: **구체적인 설명**이 부족하고, 중요한 **세부 사항**이 **누락**되었을 때 점수는 **0점**.

    ## 예시 1:
    **질문**: 코드노바의 `API 서버 기술 스택`은 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "코드노바의 API 서버는 주로 Python, Django를 사용하며, 데이터베이스는 PostgreSQL을 사용합니다. 서버는 AWS EC2에서 호스팅되며, API 게이트웨이로 Nginx를 사용합니다. 클라우드 서비스는 AWS와 Google Cloud를 활용합니다."
    **시스템 답변**: "코드노바는 Django와 PostgreSQL을 사용하여 API 서버를 운영합니다. AWS와 Google Cloud를 이용하고, Nginx로 트래픽을 관리합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요. (예: 95)
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요. (예: 90)
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요. (예: 80)

    ## 예시 2:
    **질문**: `getDistance` 메서드의 입력 매개변수는 무엇인가요?
    **기준 정답 (코드노바 사내 문서)**: "`getDistance` 메서드는 두 개의 매개변수를 받습니다: `origin` (경로의 시작점, 문자열)과 `destination` (경로의 종료점, 문자열)입니다. 이 메서드는 시작점과 종료점 간의 거리를 계산하여 반환합니다."
    **시스템 답변**: "`getDistance` 메서드는 `origin`과 `destination` 매개변수를 사용하여 거리를 계산합니다."

    ### 평가:
    - **정확도 (Accuracy)**: 점수는 0에서 100 사이로 평가하세요.
    - **재현율 (Recall)**: 점수는 0에서 100 사이로 평가하세요.
    - **구체성 (Specificity)**: 점수는 0에서 100 사이로 평가하세요.

    ## 사용자 질문 : {question}
    ## 기준 정답 : {answer}
    ## 시스템 답변 : {system_answer}

    시스템의 답변을 기반으로 정확도, 재현율, 구체성을 평가한 후, 각 항목에 대한 점수를 **0에서 100 사이**로 계산하여 다음 JSON 형태로 제공합니다:

    {{"accuracy": 85, "recall": 75, "specificity": 65}}
    """
    )

    evaluation_chain = evaluation_prompt | llm | StrOutputParser()
    return evaluation_chain


### 우리의 파인튜닝된 sllm의 RAG 답변을 배치로 정성적 평가

In [11]:
import json
import pandas as pd
import numpy as np

# JSON 응답을 받는 평가 체인 생성
evaluation_chain_json = api_document_evaluation_chain_json()

# 모든 데이터에 대해 평가 수행
evaluation_results = []

for idx in range(len(dataset_df)):
    question = dataset_df['user_input'][idx]
    answer = dataset_df['reference'][idx]
    
    # run_langraph로 시스템 답변 생성
    system_answer = chat({"history": [
    {"role": "user", "content": question}], "tone":"informal", "permission":"cto"})['response']
    
    # 평가 수행
    response = evaluation_chain_json.invoke({
        "question": question,
        "answer": answer,
        "system_answer": system_answer
    })
    
    # JSON 파싱
    try:
        scores = json.loads(response)
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': scores.get('accuracy', 0),
            'recall': scores.get('recall', 0),
            'specificity': scores.get('specificity', 0)
        })
        print(f"평가 완료: {idx+1}/{len(dataset_df)} - Accuracy: {scores.get('accuracy', 0)}")
    except json.JSONDecodeError:
        print(f"JSON 파싱 오류: {idx+1}/{len(dataset_df)}")
        evaluation_results.append({
            'index': idx,
            'question': question,
            'reference': answer,
            'system_answer': system_answer,
            'accuracy': 0,
            'recall': 0,
            'specificity': 0
        })

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(evaluation_results)

# 평균 점수 계산
avg_accuracy = results_df['accuracy'].mean()
avg_recall = results_df['recall'].mean()
avg_specificity = results_df['specificity'].mean()
overall_avg = (avg_accuracy + avg_recall + avg_specificity) / 3

print(f"\n=== 전체 평가 결과 ===")
print(f"평균 정확도: {avg_accuracy:.2f}")
print(f"평균 재현율: {avg_recall:.2f}")
print(f"평균 구체성: {avg_specificity:.2f}")
print(f"전체 평균: {overall_avg:.2f}")

# 결과 저장
results_df.to_csv('batch_evaluation_results.csv', index=False)
print(f"\n결과가 'batch_evaluation_results.csv'에 저장되었습니다.")


INFO:utils2.service:Chat Request - Permission: cto, Tone: informal
INFO:utils2.service:Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다. 질문이 들어오면 반드시 cto_search 툴을 호출하여 문서를 검색하세요.
INFO:httpx:HTTP Request: POST https://qfs7408cldsqap-8000.proxy.runpod.net/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:LLM Tool Parse Response Success
INFO:utils2.service:LLM Tools Match
INFO:utils2.service:['{"name": "cto_search", "arguments": {"keyword": "크랙(Crack) 앱의 주요 기능은 무엇인가요?"}}']
INFO:utils2.service:cto search keyword: 크랙(Crack) 앱의 주요 기능은 무엇인가요?


name='cto_search' description='모든 team(backend, frontend, data_ai)의 문서를 검색합니다.' args_schema=<class 'langchain_core.utils.pydantic.cto_search'> func=<function cto_search at 0x000001B235D6EFC0>


INFO:utils2.service:cto search found 5 results.
INFO:utils2.service:cto_search tool Response Success


검색 결과:
-----
<!-- 회사: 코드노바 | 대상: 사원(백엔드) | 작성일: 2025-08-29 -->
# 서비스 아키텍처 문서
분류: backend | 회사: 코드노바 | 버전: v1.0 | 작성일: 2025-08-29

## 1. 개요
코드노바의 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI 페르소나 챗봇 앱인 크랙(Crack), 그리고 대화형 광고 제작·보상 플랫폼인 Wrtn Ads를 지원하도록 설계되었습니다. 이 문서는 백엔드 시스템의 구성 요소와 상호작용을 설명합니다.

## 2. 아키텍처 구성 요소

### 2.1. API 서버
- **역할**: 클라이언트와의 통신을 담당하며, 요청을 처리하고 적절한 응답을 반환합니다.
- **기술 스택**: Node.js, Express.js
- **검증 포인트**:
  - API 엔드포인트가 올바르게 작동하는지 확인
  - 요청 처리 속도 및 오류율 모니터링 [[ref1]]
## 6. 의존성 관리
- **패키지 업데이트**: 사용하는 라이브러리와 패키지는 정기적으로 업데이트하여 최신 보안 패치를 적용합니다.
- **취약점 스캐닝**: 의존성에 대한 취약점 스캐닝 도구를 사용하여 보안 취약점을 사전 예방적으로 점검합니다.

## 7. 오류 처리
- **사용자 친화적인 오류 메시지**: 오류 발생 시, 사용자에게는 일반적인 오류 메시지만 표시하고, 내부 정보는 노출하지 않도록 합니다.
- **로깅 및 모니터링**: 오류 발생 시, 로그를 기록하고, 이를 통해 보안 사고를 추적할 수 있도록 합니다.

## 8. 배포 보안
- **배포 환경 설정**: 배포 환경은 개발 환경과 분리하여 설정하고, 필요한 경우 방화벽을 설정합니다.
- **CI/CD 파이프라인 보안**: CI/CD 파이프라인의 접근 권한을 제한하고, 보안 점검을 자동화하여 배포 전 검증합니다. [[ref2]]
### 2.2. 데이터베이스
- **역할**: 사용자 데이터, 콘텐츠, 로그 등을 저장합니다.
- **기술 스택**: Po

INFO:httpx:HTTP Request: POST https://qfs7408cldsqap-8000.proxy.runpod.net/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:Final Assistant Reply Generated
INFO:utils2.service:크랙(Crack) 앱의 주요 기능은 생성형 AI를 활용한 글쓰기, 이미지 생성, 요약 플랫폼을 제공하는 것입니다. 또한, AI 페르소나 챗봇 앱으로서 대화형 광고 제작 및 보상 플랫폼을 지원하는 기능도 포함되어 있습니다.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:Chat Request - Permission: cto, Tone: informal
INFO:utils2.service:Tools Prompt: 사용자는 cto로서, 모든 팀의 문서를 열람할 수 있는 개발팀 최고 관리자입니다. 질문이 들어오면 반드시 cto_search 툴을 호출하여 문서를 검색하세요.


...

name='cto_search' description='모든 team(backend, frontend, data_ai)의 문서를 검색합니다.' args_schema=<class 'langchain_core.utils.pydantic.cto_search'> func=<function cto_search at 0x000001B235D6EFC0>


INFO:utils2.service:cto search found 5 results.
INFO:utils2.service:cto_search tool Response Success


검색 결과:
-----
## 6. 개정 이력
- v1.0 — 2025-08-29: 최초 작성

--- 

이 문서는 데이터/AI팀의 정기회의 내용을 바탕으로 작성되었습니다. 각 팀원은 Action Item에 따라 책임을 다하고, 정기적으로 진행 상황을 업데이트해 주시기 바랍니다. [[ref1]]
### 3.1 변경 이력 기록 양식

- **변경 날짜**: YYYY-MM-DD
- **변경 사항**: 구체적인 변경 내용
- **작성자**: 변경을 수행한 개발자 이름

### 3.2 예시

- **변경 날짜**: 2025-08-01
- **변경 사항**: 사용자 테이블에 'last_login' 필드 추가
- **작성자**: 홍길동

## 4. 검증 포인트

데이터베이스 스키마 변경 후 다음 사항을 검증합니다:

- 모든 테이블과 필드가 올바르게 생성되었는지 확인
- 외래 키 제약 조건이 제대로 설정되었는지 확인
- 기존 데이터와의 호환성 검증
- 쿼리 성능 테스트 수행

## 5. 결론

본 문서는 코드노바의 데이터베이스 스키마 및 변경 이력에 대한 기본 정보를 제공합니다. 향후 변경 사항이 있을 경우, 반드시 변경 이력을 기록하고 검증 포인트를 확인하여 데이터의 무결성을 유지해야 합니다.

--- [[ref2]]
## 7. 개정 이력
- v1.0 — 2025-08-29: 초기 버전 작성

--- 

이 문서는 데이터/AI팀이 당장 실행 가능한 단계와 체크리스트를 포함하고 있습니다. 각 단계별로 필요한 조치를 취하여 모델 성능을 최적화하시기 바랍니다. [[ref3]]
---

## 1. 이번 주 최우선 과제
| 목표                     | 성공 기준                             | 담당자      | 마감일     |
|--------------------------|--------------------------------------|-------------|------------|
| 모델 성능 개선           | A

INFO:httpx:HTTP Request: POST https://qfs7408cldsqap-8000.proxy.runpod.net/v1/chat/completions "HTTP/1.1 200 OK"
INFO:utils2.service:Final Assistant Reply Generated
INFO:utils2.service:2025-08-01에 기록된 변경 사항은 사용자 테이블에 'last_login' 필드 추가입니다. 해당 변경 사항은 홍길동이 작성한 것입니다.
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


평가 완료: 12/12 - Accuracy: 95

=== 전체 평가 결과 ===
평균 정확도: 77.50
평균 재현율: 76.25
평균 구체성: 73.75
전체 평균: 75.83

결과가 'batch_evaluation_results.csv'에 저장되었습니다.


In [12]:
result = pd.read_csv('batch_evaluation_results.csv')

display(result)

,index,question,reference,system_answer,accuracy,recall,specificity
0,0,크랙(Crack) 앱의 주요 기능은 무엇인가요?,"크랙(Crack)은 AI 페르소나 챗봇 앱으로, 코드노바의 서비스 아키텍처에서 생성...","크랙(Crack) 앱의 주요 기능은 생성형 AI를 활용한 글쓰기, 이미지 생성, 요...",85,80,70
1,1,RabbitMQ의 역할은 무엇인가요?,RabbitMQ는 비동기 작업 처리를 위해 시스템 간 메시지를 전달하는 메시지 큐입니다.,"RabbitMQ는 메시지 큐로, 비동기 작업 처리를 위해 시스템 간 메시지를 전달하...",90,95,85
2,2,프론트엔드 팀에서 JWT의 역할은 무엇인가요?,JWT(JSON Web Token)는 사용자 인증을 처리하는 데 사용됩니다.,프론트엔드 팀에서 JWT의 역할에 대한 구체적인 정보는 문서에서 찾을 수 없습니다....,0,0,0
3,3,"코드노바의 백엔드 서비스 아키텍처는 어떻게 설계되었고, 그 목적은 무엇인가요?",코드노바의 백엔드 서비스 아키텍처는 확장 가능하고 안정적인 서비스를 제공하기 위해 ...,"코드노바의 백엔드 서비스 아키텍처는 생성형 AI 글쓰기·이미지·요약 플랫폼, AI ...",70,60,50
4,4,MFA는 뭐하는거야?,MFA는 추가적인 보안 강화를 위해 도입되는 다중 인증 방법입니다.,"MFA는 다중 인증(Multi-Factor Authentication)의 약자로, ...",90,95,85
5,5,JWT는 API 접근 시 어떤 역할을 합니까?,"JWT(JSON Web Token)는 API 접근 시 인증을 위해 사용되며, 토큰은...",JWT(JSON Web Token)는 API 접근 시 사용자의 인증을 처리하는 역할...,95,100,90
6,6,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 무엇입니까?,이 가이드는 코드노바의 백엔드 개발팀이 보안 및 인증을 효과적으로 관리하기 위한 기...,코드노바의 백엔드 개발팀이 보안 및 인증을 관리하기 위한 기본 지침은 다음과 같습니...,70,60,80
7,7,2025년 8월 29일에 작성된 코드노바의 에러 핸들링 매뉴얼에서 에러 핸들링의 중...,코드노바의 에러 핸들링 매뉴얼에서는 에러 핸들링이 안정적인 시스템 운영을 위해 필수...,2025년 8월 29일에 작성된 코드노바의 에러 핸들링 매뉴얼에서는 에러 핸들링의 ...,95,95,90
8,8,v1.0 뭐에요?,"v1.0은 코드노바 백엔드팀의 배포 및 운영 가이드의 버전 번호로, 이 문서는 안정...","검색 결과에 따르면, v1.0은 문서의 초기 버전을 의미합니다. 구체적으로는 202...",70,60,50
9,9,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,코드노바 백엔드팀이 배포 및 운영을 효율적으로 수행하기 위해 필요한 체크리스트 항목...,100,100,100
